In [ ]:
from common_packages import *
from dataset import *
from network import *
from misc import *
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
gpu_devices = list(np.arange(torch.cuda.device_count()))
multi_gpu = len(gpu_devices) > 1
ckpt_root = './ckpt'
data_root = '../data'
output_root = 'XXX'

In [ ]:
net = CoADNet_Dilated_ResNet50(mode='test', compute_loss=False).cuda()
resume_params = os.path.join(ckpt_root, 'trained', 'CoADNet_DRN50_COCO-SEG_Final.pth')
net.load_state_dict(torch.load(resume_params))
test_set_list = ['Cosal2015', 'iCoseg', 'MSRC', 'CoSOD3k']
for test_set in test_set_list:
    output_folder = os.path.join(output_root, test_set)
    is_shuffle = False
    M = 5
    stat_file = 'Stat-' + test_set + '.txt'
    list_file = 'List-' + test_set + '.txt'
    GroupNumbers, GroupNames, GroupSizes, GroupFileLists = Group_Stat_Parser(data_root, stat_file, list_file)
    # add to integral multiple of M
    for g in range(GroupNumbers):
        group_file_list = GroupFileLists[g]
        if is_shuffle:
            np.random.shuffle(group_file_list)
        gs = len(group_file_list)
        if np.mod(gs, M) != 0:
            num_add = M - np.mod(gs, M)
            for n in range(num_add):
                group_file_list.append(group_file_list[n])
            GroupSizes[g] += num_add
    net.eval()
    for group_id in range(GroupNumbers):
        print('{}:{}'.format(test_set, align_number(group_id+1, 3)))
        group_file_list = GroupFileLists[group_id]
        for index in range(0, len(group_file_list), M):
            images_bag = []
            names_bag = []
            for j in range(index, index + M):
                ti = transforms.ToTensor()
                images_bag.append(ti(Image.open(os.path.join(data_root, group_file_list[j])+'.jpg')).unsqueeze(0))
                names_bag.append(group_file_list[j].split('/')[-1])
            with torch.no_grad():
                grp_images = torch.cat(images_bag, dim=0).unsqueeze(0).cuda()
                cosod_maps = net(grp_images)
            for m in range(M):
                group_name = names_bag[m].split('__')[0]
                save_smap(cosod_maps[0, m, ...], os.path.join(output_folder, group_name, names_bag[m] + '.png'))
    torch.cuda.empty_cache()
    print('finished testing over: {}'.format(test_set))